In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import json
import random
import numpy as np
import pandas as pd
import pickle
import sys
import torchinfo

import torch 
from torch import nn 
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import itertools
import random
import copy
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import cv2
import json
from glob import glob
from sklearn.model_selection import train_test_split
from functools import partial
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from itertools import product
import matplotlib.pyplot as plt
from sklearn import metrics
from tabulate import tabulate
import math
import logging
from datetime import datetime
from sklearn.metrics import accuracy_score
import argparse

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def classname_id(class_name_list):
    id2classname = {k:v for k, v in zip(list(range(len(class_name_list))),class_name_list)}
    classname2id = {v:k for k, v in id2classname.items()}
    return id2classname, classname2id

def trunc(latent, mean_size, truncation):  # Truncation trick on Z
    t = Variable(FloatTensor(np.random.normal(0, 1, (mean_size, *latent.shape[1:]))))
    m = t.mean(0, keepdim=True)

    for i,_ in enumerate(latent):
        latent[i] = m + truncation*(latent[i] - m)

    return latent

In [7]:
from torch.autograd import Variable
import torch

from utils import general
from collections import Counter
import pickle

In [8]:
config = {
    "batch_size":10,
    "latent_dim":512,
    "mlp_dim":8,
    "n_classes":120,
    "label":-1,
    "t_size":64,
    "v_size":25,
    "channels":3,
    "dataset":"ntu",
    "model":"/content/model_saves/generator_ntu120_xsub_mlp8_2150000.pth",
    "stochastic":False,
    "stochastic_file":"-",
    "stochastic_index":0,
    "gen_qtd":10,
    "trunc":0.95,
    "trunc_mode":'w',
    "mean_size":1000
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
class Generator(nn.Module):
    def __init__(self,latent_dim):
        super(Generator, self).__init__()
        self.latent_dim = 1024

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(self.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img

device(type='cpu')

In [11]:
generator     = Generator(
    1024, 
    2,
    60, 
    mlp_dim=4, 
    dataset="ntu",
    device="cpu"
    ).to(device)

In [12]:
generator(torch.rand((32,1024)).to(device)).shape

torch.Size([32, 2, 60, 25])

In [13]:
torchinfo.summary(generator, input_size=(32,1024), col_names = ("input_size", "output_size", "num_params", "kernel_size", "mult_adds"))

/usr/local/lib/python3.10/dist-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/usr/local/lib/python3.10/dist-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Generator                                     [32, 1024]                [32, 2, 60, 25]           2,757                     --                        --
├─Mapping_Net: 1-1                            [1024]                    [1024]                    --                        --                        --
│    └─Sequential: 2-1                        [1024]                    [1024]                    --                        --                        --
│    │    └─Linear: 3-1                       [1024]                    [1024]                    1,049,600                 --                        1,074,790,400
│    │    └─LeakyReLU: 3-2                    [1024]                    [1024]                    --                        --                        --
│    │    └─Linear: 3-3                       [1024]            

# *Below Code WorthLess Don't Check* 

In [ ]:
#out = general.check_runs('kinetic-gan', id=-1)
out = "/content/runs/kinetic-gan"
actions_out = os.path.join(out, 'actions')
if not os.path.exists(actions_out): os.makedirs(actions_out)

config_file = open(os.path.join(out,"gen_config.txt"),"w")
config_file.write("Kinetic-GAN.ipynb" + '|' + str(config))
config_file.close()

cuda = True if torch.cuda.is_available() else False
print(cuda)

# Initialize generator 
generator     = Generator(
    config["latent_dim"], 
    config["channels"], 
    config["n_classes"], 
    config["t_size"], 
    mlp_dim=config["mlp_dim"], 
    dataset=config["dataset"])

if cuda:
    generator.cuda()

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

# Load Models
generator.load_state_dict(torch.load(config["model"], map_location=device),strict=False)
generator.eval()

new_imgs   = []
new_labels = []
z_s        = []

classes = np.arange(config["n_classes"]) if config["label"] == -1 else [config["label"]]
qtd = config["batch_size"]

if config["stochastic_file"]!='-':
    stoch = np.load(config["stochastic_file"]) 
    stoch = np.expand_dims(stoch[config["stochastic_index"]], 0)
    print(stoch.shape)

if config["stochastic"]:  # Generate one latent point 
    z   = Variable(FloatTensor(np.random.normal(0, 1, (1, config["latent_dim"]))  if config["stochastic_file"] == '-' else stoch ))
    z   = z.repeat(qtd*len(classes),1)

while(len(classes)>0):

    if not config["stochastic"]: # Generate Samples if not in mode stochastic
        z         = Variable(FloatTensor(np.random.normal(0, 1, (qtd*len(classes), config["latent_dim"])))) 

    z         = trunc(z, config["mean_size"], config["trunc"]) if config["trunc_mode"]=='z' else z
    labels_np = np.array([num for _ in range(qtd) for num in classes])  # Generate labels
    labels    = Variable(LongTensor(labels_np))
    gen_imgs  = generator(z, labels, config["trunc"]) if config["trunc_mode"] == 'w' else generator(z, labels)

    new_imgs   = gen_imgs.data.cpu()  if len(new_imgs)==0 else np.concatenate((new_imgs, gen_imgs.data.cpu()), axis=0)
    new_labels = labels_np if len(new_labels)==0 else np.concatenate((new_labels, labels_np), axis=0)
    z_s        = z.cpu()  if len(z_s)==0 else np.concatenate((z_s, z.cpu()), axis=0)   
    

    tmp     = Counter(new_labels)
    classes = [i for i in classes if tmp[i]<config["gen_qtd"]]

    print('---------------------------------------------------')
    print(tmp)
    print(len(new_labels), classes)


if config["dataset"] == 'ntu':
    new_imgs = np.expand_dims(new_imgs, axis=-1)
    

new_labels = np.concatenate((np.expand_dims(new_labels, 0), np.expand_dims(new_labels, 0)), axis=0)  # Remove if not needed

with open(os.path.join(actions_out, str(config["n_classes"] if config["label"] == -1 else config["label"])+'_'+str(config["gen_qtd"])+('_trunc' + str(config["trunc"]) if config["trunc_mode"]!='-' else '')+('_stochastic' if config["stochastic"] else '')+'_gen_data.npy'), 'wb') as npf:
    np.save(npf, new_imgs)


with open(os.path.join(actions_out, str(config["n_classes"] if config["label"] == -1 else config["label"])+'_'+str(config["gen_qtd"])+('_trunc' + str(config["trunc"]) if config["trunc_mode"]!='-' else '')+('_stochastic' if config["stochastic"] else '')+'_gen_z.npy'), 'wb') as npf:
    np.save(npf, z_s)


with open(os.path.join(actions_out, str(config["n_classes"] if config["label"] == -1 else config["label"])+'_'+str(config["gen_qtd"])+('_trunc' + str(config["trunc"]) if config["trunc_mode"]!='-' else '')+('_stochastic' if config["stochastic"] else '')+'_gen_label.pkl'), 'wb') as f:
    pickle.dump(new_labels, f)

In [ ]:
data = np.load("/content/runs/kinetic-gan/actions/120_10_trunc0.95_gen_data.npy")

with open("/content/runs/kinetic-gan/actions/120_10_trunc0.95_gen_label.pkl", 'rb') as f:
    labels = pickle.load(f)

In [ ]:
data = np.squeeze(data)
labels = labels[0].squeeze()

In [ ]:
data.shape,labels.shape

((1200, 3, 64, 25), (1200,))

In [ ]:
#!rm -r runs/synthetic

In [ ]:
#!python visualization/action_ntu.py --path "/content/runs/kinetic-gan/actions/120_10_trunc0.95_gen_data.npy" --indexes 26 86 146 

In [ ]:
def gen_skeleton(frame, 
                 height,
                 width,
                 mapping_list = [(0, 1), (1, 3), (3, 5), 
                                 (0, 2), (2, 4), (0, 6), 
                                 (1, 7), (6, 7), (6, 8), 
                                 (7, 9), (8, 10), (9, 11)]):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3

def gen_video(points, 
              save_file, 
              frame_h, 
              frame_w, 
              is_3d=True,
              mapping_list = [(0, 1), (1, 3), (3, 5), 
                                 (0, 2), (2, 4), (0, 6), 
                                 (1, 7), (6, 7), (6, 8), 
                                 (7, 9), (8, 10), (9, 11)]):
    # make 3D if points are flatten
    if len(points.shape) == 2:
        if is_3d:
          fts = points.shape[1]
          x_cds = list(range(0, fts, 3))
          y_cds = list(range(1, fts, 3))
          z_cds = list(range(2, fts, 3))
          points = np.transpose(np.array([points[:, x_cds], 
                                          points[:, y_cds], 
                                          points[:, z_cds]]), (1,2,0))
        else:
          fts = points.shape[1]
          x_cds = list(range(0, fts, 2))
          y_cds = list(range(1, fts, 2))
          points = np.transpose(np.array([points[:, x_cds], 
                                          points[:, y_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    for __id,frame in enumerate(points):
        skel_image = gen_skeleton(frame, frame_h, frame_w,mapping_list=mapping_list)
        result.write(skel_image)

    result.release()

In [ ]:
joint_map = [(3,2),(2,20),(20,4),(4,5),(5,6),(6,7),(7,21),(7,22),(20,8),(8,9),(9,10),(10,11),(11,23),(11,24),
            (20,1),(1,0),(0,12),(12,13),(13,14),(14,15),(0,16),(16,17),(17,18),(18,19),(8,16),(4,12),(8,4),(16,12)]

In [ ]:
save_vids_dir = "checking_vids/init"
for index,adata,alabel in enumerate(tqdm(zip(data,labels))):
  data = adata
  file_id = index
  target = alabel
  vid_size = [int(adata[3][0][selected_ind]),int(adata[3][1][selected_ind])]

  try:
    if not os.path.exists(f"{save_vids_dir}/{file_id}/dataloader_out_cls_{target}.mp4"):
      os.makedirs(f"{save_vids_dir}/{file_id}",exist_ok=True)
      gen_video(data, 
                f"{save_vids_dir}/{file_id}/dataloader_out_cls_{target}.mp4",
                vid_size[0], 
                vid_size[1],
                is_3d=False,
                mapping_list=joint_map
                )
  except ValueError:
    continue

torch.Size([32, 2, 60, 25])